### Initialize Timeplus

In [ ]:
import os
import json
import pandas as pd
from pprint import pprint
from perspective import Table, PerspectiveWidget
from timeplus import Query, Environment

api_key = "apikey"
api_address = "https://dev.timeplus.cloud"
workspace = "tp-demo"

# Configure API key and address
env = Environment().address(api_address).workspace(workspace).apikey(api_key)

### Run a timeplus query

run a stream query and show the returned cols

In [ ]:
sql = '''
SELECT 
  *
FROM 
  car_live_data
'''

query = (
        Query(env=env).sql(query=sql)
        .create()
    )
query_header = query.header()
pprint(query_header)

### Show perspective table

build a widget using the first returned batch data

using `cid` as the index 

In [ ]:
columns = [ f['name'] for f in query_header]

first_row = next(query.result())
if first_row.event == "message":
    data = json.loads(first_row.data)
    df = pd.DataFrame(data, columns=columns)
    
table = Table(df, index="cid")
PerspectiveWidget(table, plugin="Y Line", columns=['speed_kmh'])

### Update table 

update the table, the data is directly binding to the widget

In [ ]:
for event in query.result():
    if event.event == "message":
        new_data = json.loads(event.data)
        update_data = {}
        for col in columns:
            update_data[col] = []
        for row in new_data:
            for index, col in enumerate(columns):
                update_data[col].append(row[index])   
        table.update(update_data)